# Implementation of Navier-Stokes equation using fenicsx/dolfinx
## Steps to Navier-Stokes using Chorin's projection method
Momentum: $\rho \frac{D u}{Dt} = -\nabla p + \mu \nabla^2 u + \rho \mathbf{g}$

Continuity: $\nabla \cdot u = 0$
1. Taylor-Hood elements are used to avoid the saddle point problem. Pressure finite element space is one order lower than the velocity finite element space.

2. Solve the weak form of the momentum equation without the pressure gradient term. Convection is treated explicitly and diffusion is treated implictly. Tentative velocity step (solve without pressure gradient term).
$$0 = \frac{1}{\Delta t} \langle u^* - u, v \rangle +  \langle (\nabla u)u, v \rangle + \nu \langle \nabla u^* - \nabla v \rangle$$

3. Pressure Poisson solution to generate correct pressure
$$\langle \nabla p^*, \nabla q\rangle = -\frac{1}{\Delta t}\langle \nabla \cdot u^*,q \rangle$$

4. Final velocity update using the Chorin projection
$$ \langle u^{**},v \rangle  = \langle u^*, v \rangle - \frac{1}{\Delta t}\langle \nabla p^*, v \rangle$$


## Lid driven cavity
Lid driven cavity is a standard Drichilet problem solved on a square grid. The upper lid moves in the x direction with a fixed velocity and the generated solution has a swirling nature. The location of the center of the swirl is a function of the Reynolds number.

### Initial condition
Velocity components are zero everywhere. Homogeneous Dirichlet BC everywhere except the top. No applied volume forces.

### Glossary of variables
* $u$   (`u_prev`): Velocity field from previous iteration
* $u^*$  (`u_tent`): Velocity field after tentative momentum step
* $u^{**}$ (`u_next`): Velocity field corrected for incompressibility constraint
* $p^*$  (`p_next`): Pressure field at the next step
* $\Delta t$ (`dt`): Time step size
* v (`v`): Test function for u
* q (`q`): Test function for p

In [1]:
# Importing the standard libraries
from mpi4py import MPI
from petsc4py import PETSc
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import dolfinx as fe
import ufl

In [2]:
N_POINTS_P_AXIS = 41

TIME_STEP_LENGTH = 0.01
N_TIME_STEPS = 100
KINEMATIC_VISCOSITY = 0.01 # -> Re = 100

In [3]:
mesh = fe.mesh.create_unit_square(MPI.COMM_WORLD,N_POINTS_P_AXIS, N_POINTS_P_AXIS)

# Taylor-Hood Elements. The order of the function space for the pressure has
# to be one order lower than for the velocity. Elements are defined 
velocity_element = ufl.VectorElement('CG',mesh.ufl_cell(), 2) # CG stands for Continuous-Galerkin
pressure_element = ufl.FiniteElement('CG',mesh.ufl_cell(), 1)

velocity_function_space = fe.fem.FunctionSpace(mesh, velocity_element)
pressure_function_space = fe.fem.FunctionSpace(mesh, pressure_element)

u_trial = ufl.TrialFunction(velocity_function_space)
p_trial = ufl.TrialFunction(pressure_function_space)
v_test = ufl.TestFunction(velocity_function_space)
q_test = ufl.TestFunction(pressure_function_space)

In [4]:
# Define the Boundary Condition
def stationary_walls(x):
    return np.logical_or(np.isclose(x[0],0), np.isclose(x[0],1),np.isclose(x[1],0))

stationary_walls_dofs = fe.fem.locate_dofs_geometrical(
    velocity_function_space, 
    stationary_walls)

u_noslip = np.array(
    (0,) * mesh.geometry.dim,
    dtype=PETSc.ScalarType)

stationary_wall_boundary_condition = fe.fem.dirichletbc(
    u_noslip,
    stationary_walls_dofs,
    velocity_function_space)

def moving_wall(x):
    return np.logical_or(np.isclose(x[1],1))

moving_walls_dofs = fe.fem.locate_dofs_geometrical(
    velocity_function_space, 
    stationary_walls)
u_moving = np.array(
    (1,) * mesh.geometry.dim,
    dtype=PETSc.ScalarType)
moving_wall_boundary_condition = fe.fem.dirichletbc(
    u_moving,
    stationary_walls_dofs,
    velocity_function_space)

# Vector containing the defined BCs
velocity_boundary_conditions = [stationary_wall_boundary_condition, moving_wall_boundary_condition]

In [23]:
# Define the solution fields involved
u_prev = fe.fem.Function(velocity_function_space)
u_tent = fe.fem.Function(velocity_function_space)
u_next = fe.fem.Function(velocity_function_space)
p_next = fe.fem.Function(pressure_function_space)

# Weak form of the momentum equation
momentum_weak_form_residuum = (
    1.0 / TIME_STEP_LENGTH * ufl.inner(u_trial - u_prev, v_test) * ufl.dx
    +
    ufl.inner(ufl.nabla_grad(u_prev) * u_prev, v_test) * ufl.dx
    +
    KINEMATIC_VISCOSITY * ufl.inner(ufl.nabla_grad(u_trial), ufl.nabla_grad(v_test)) * ufl.dx
)
momentum_weak_form_lhs = fe.fem.form(ufl.lhs(momentum_weak_form_residuum))
momentum_weak_form_rhs = fe.fem.form(ufl.rhs(momentum_weak_form_residuum))

# Weak form of the pressure poisson problem
pressure_poisson_weak_form_lhs = fe.fem.form(ufl.inner(ufl.nabla_grad(p_trial), ufl.nabla_grad(q_test)) * ufl.dx)
pressure_poisson_weak_form_rhs = fe.fem.form(- 1.0 / TIME_STEP_LENGTH * ufl.nabla_div(u_tent) * q_test * ufl.dx)

# Weak form of the velocity update equation
velocity_update_weak_form_lhs = fe.fem.form(ufl.inner(u_trial, v_test) * ufl.dx)
velocity_update_weak_form_rhs = fe.fem.form(
    ufl.inner(u_tent, v_test) * ufl.dx
    -
    TIME_STEP_LENGTH * ufl.inner(ufl.nabla_grad(p_next), v_test) * ufl.dx
)

# Pre-Compute the system matrices (because they do not greatly change)
momentum_assembled_system_matrix = fe.fem.petsc.assemble_matrix(momentum_weak_form_lhs, bcs=velocity_boundary_conditions)
momentum_assembled_system_matrix.assemble()
pressure_poisson_assembled_system_matrix = fe.fem.petsc.assemble_matrix(pressure_poisson_weak_form_lhs)
pressure_poisson_assembled_system_matrix.assemble()
velocity_update_assembled_system_matrix = fe.fem.petsc.assemble_matrix(velocity_update_weak_form_lhs)
velocity_update_assembled_system_matrix.assemble()

In [24]:
# Solver for step 1
solver1 = PETSc.KSP().create(mesh.comm)
solver1.setOperators(momentum_assembled_system_matrix)
solver1.setType(PETSc.KSP.Type.BCGS)
pc1 = solver1.getPC()
pc1.setType(PETSc.PC.Type.HYPRE)
pc1.setHYPREType("boomeramg")

# Solver for step 2
solver2 = PETSc.KSP().create(mesh.comm)
solver2.setOperators(pressure_poisson_assembled_system_matrix)
solver2.setType(PETSc.KSP.Type.BCGS)
pc2 = solver2.getPC()
pc2.setType(PETSc.PC.Type.HYPRE)
pc2.setHYPREType("boomeramg")

# Solver for step 3
solver3 = PETSc.KSP().create(mesh.comm)
solver3.setOperators(velocity_update_assembled_system_matrix)
solver3.setType(PETSc.KSP.Type.CG)
pc3 = solver3.getPC()
pc3.setType(PETSc.PC.Type.SOR)

In [19]:
for t in tqdm(range(N_TIME_STEPS)):
    # (1) Solve for tentative velocity
    momentum_assembled_rhs = fe.fem.petsc.assemble_vector(momentum_weak_form_rhs)

        
#    [bc.apply(momentum_assembled_system_matrix, momentum_assembled_rhs) for bc in velocity_boundary_conditions]
    fe.fem.solve(momentum_weak_form_lhs == momentum_weak_form_rhs, u_prev.vector(), bcs=velocity_boundary_conditions)

    # (2) Solve for the pressure
    pressure_poisson_assembled_rhs = fe.fem.petsc.assemble_vector(pressure_poisson_weak_form_rhs)
    fe.solve(
        pressure_poisson_assembled_system_matrix,
        p_next.vector(),
        pressure_poisson_assembled_rhs,
        "gmres",
        "amg",
    )

    # (3) Correct the velocities to be incompressible
    [bc.apply(momentum_assembled_system_matrix, momentum_assembled_rhs) for bc in velocity_boundary_conditions]
    velocity_update_assembled_rhs = fe.fem.petsc.assemble_vector(velocity_update_weak_form_rhs)
    fe.solve(
        velocity_update_assembled_system_matrix,
        u_next.vector(),
        velocity_update_assembled_rhs,
        "gmres",
        "ilu",
    )

    # Advance in time
    u_prev.assign(u_next)

    # Visualize interactively
    fe.plot(u_next)
    plt.draw()
    plt.pause(0.02)
    plt.clf()

  0%|                                                   | 0/100 [00:00<?, ?it/s]


TypeError: create_matrix(): incompatible function arguments. The following argument types are supported:
    1. (a: dolfinx::fem::Form<double>, type: str = '') -> mat

Invoked with: <petsc4py.PETSc.Mat object at 0x11e66f6a0>